In [1]:
import ujson
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import multiprocessing
from copy import deepcopy

from itertools import ifilter


pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [3]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

# for _line in log_progress(lines,every=1):
#     _line = _line.strip()

time: 49.1 ms


In [5]:
with open("/Users/patrickmulrooney/Downloads/pmulrooney_run.json", 'r') as f:
    # Just testing the import, runs out of memory
    # TODO: Figure out why or how to deal with.
    endo = ujson.load(f)

time: 4.94 ms


In [11]:
for _i in sorted(endo.keys()):
    if type(endo[_i]) not in (dict, list):
        print _i, ':', endo[_i]
    else:
        print _i, ':', type(endo[_i])

admin_rejected : False
altitude_max : 6.6
altitude_min : -9.7
ascent : 0.0
author : <type 'dict'>
calories : 1231.6
can_copy : True
descent : 10.0
distance : 16.0960006714
duration : 4844.59
expand : full
feed_id : 281475874731619
hashtags : <type 'list'>
hydration : 0.89
id : 871971752
include_in_stats : True
is_live : False
is_peptalk_allowed : False
laps : <type 'dict'>
local_start_time : 2017-02-12T14:54:49.000-05:00
pb_count : 8
personal_bests : <type 'list'>
pictures : <type 'list'>
points : <type 'dict'>
records : <type 'list'>
show_map : 0
show_workout : 0
small_encoded_polyline : ch~fEpaijUDiAfDkAxArFdD`l@pNz{ClD|ToBxHoBIcAoE_MdCqIpP_GvD{ExM{RzY|ApDgBnAcEzMpGjFtGmC|KhEhKNxKcDdIiGgAmHyFwA\uCnCEzDvDjApPYrLwBeAyHtG_TTy@[CsLeGm@uE{CyEvCuBQ_EqD^oEgBgDn\uh@tE}L|CGxK{Q@qBtLaB`AjE`CCbAy@T}G_CwJi@mWsO}{Bt@wG{Cwm@m@wGsA{Cw@t@
speed_avg : 11.96088883
speed_max : 16.7816
sport : 0
start_time : 2017-02-12T19:54:49.000Z
tagged_users : <type 'list'>
weather : <type 'dict'>
time: 10.3 ms


In [25]:
cum_dist = 0.0
duration = 0.0
for _i in endo['points']['points']:
    cum_dist += _i['distance']
    duration += _i['duration']


time: 3 ms


In [27]:
print cum_dist
print duration

4069.987
1217768000.0
time: 2.04 ms


In [24]:
for _i in endo['points']['points']:
    print _i
    break

{u'distance': 0.0, u'sensor_data': {}, u'instruction': 2, u'longitude': -117.1972076, u'duration': 0.0, u'time': u'2017-02-12T19:54:49.000Z', u'latitude': 32.7643452}
time: 3.1 ms


In [31]:
endo['points']['points'][199:202]

[{u'altitude': -3.7,
  u'distance': 6.564,
  u'duration': 1953000.0,
  u'latitude': 32.767169,
  u'longitude': -117.2497195,
  u'sensor_data': {u'speed': 13.0414},
  u'speed': 13.0414,
  u'time': u'2017-02-12T20:27:22.000Z'},
 {u'altitude': -3.6,
  u'distance': 6.583,
  u'duration': 1959000.0,
  u'latitude': 32.7670118,
  u'longitude': -117.2496973,
  u'sensor_data': {u'speed': 10.9308},
  u'speed': 10.9308,
  u'time': u'2017-02-12T20:27:28.000Z'},
 {u'altitude': -3.6,
  u'distance': 6.611,
  u'duration': 1967000.0,
  u'latitude': 32.7668032,
  u'longitude': -117.2498716,
  u'sensor_data': {u'speed': 11.0243},
  u'speed': 11.0243,
  u'time': u'2017-02-12T20:27:36.000Z'}]

time: 9.15 ms


In [33]:
import math

math.sqrt((32.7668032 - 32.7670118)**2 + (-117.2498716+117.2496973)**2)

0.0002718353361952054

time: 4.94 ms


In [37]:
endo['points']['points'][:3]

[{u'distance': 0.0,
  u'duration': 0.0,
  u'instruction': 2,
  u'latitude': 32.7643452,
  u'longitude': -117.1972076,
  u'sensor_data': {},
  u'time': u'2017-02-12T19:54:49.000Z'},
 {u'altitude': 2.0,
  u'distance': 0.019,
  u'duration': 21000.0,
  u'latitude': 32.7643371,
  u'longitude': -117.1970196,
  u'sensor_data': {u'speed': 6.7593},
  u'speed': 6.7593,
  u'time': u'2017-02-12T19:55:10.000Z'},
 {u'altitude': 1.9,
  u'distance': 0.037,
  u'duration': 27000.0,
  u'latitude': 32.7643108,
  u'longitude': -117.1968344,
  u'sensor_data': {u'speed': 11.3106},
  u'speed': 11.3106,
  u'time': u'2017-02-12T19:55:16.000Z'}]

time: 4.89 ms


In [38]:
len(endo['points']['points'])

500

time: 3.52 ms


```
$ curl https://www.endomondo.com/rest/v1/users/1860957
{"expand":"full","id":1860957,"first_name":"Britt","middle_name":"Drud","last_name":"Sørensen","picture":{"url":"https://www.endomondo.com/resources/gfx/image/39389457/eb54be7526bac56b9a64a2d85279f25c/thumbnail.jpg"},"gender":1,"is_premium":true,"height":168.0,"hr_zones":{"max":200,"rest":60,"zone1_start":130,"zone2_start":144,"zone3_start":158,"zone4_start":172,"zone5_start":186},"date_of_birth":"1986-02-15T11:00:00.000Z","workout_count":1806,"favorite_sport":0,"friend_count":69,"summary":{"count":1806,"total_distance":8764.835977889446,"total_duration":6244217.983999999,"total_calories":676949.0},"summary_by_sport":[{"sport":0,"count":1358,"total_distance":6834.78174495984,"total_duration":2775496.0399999986,"total_calories":476243.0},{"sport":18,"count":263,"total_distance":1242.4142599254849,"total_duration":3033149.0,"total_calories":148514.0},{"sport":2,"count":127,"total_distance":533.3600199999998,"total_duration":206750.94399999996,"total_calories":21217.0},{"sport":19,"count":41,"total_distance":137.27995300412178,"total_duration":145122.0,"total_calories":19626.5},{"sport":22,"count":10,"total_distance":0.0,"total_duration":31200.0,"total_calories":4637.66},{"sport":87,"count":9,"total_distance":0.0,"total_duration":29400.0,"total_calories":4370.1},{"sport":9,"count":5,"total_distance":16.0,"total_duration":49500.0,"total_calories":6131.52},{"sport":20,"count":2,"total_distance":1.0,"total_duration":3000.0,"total_calories":578.891},{"sport":46,"count":1,"total_distance":0.0,"total_duration":1800.0,"total_calories":267.557}],"created_date":"2011-06-26T23:14:14.000Z","has_dismissed_download_box":true,"country":"DK","personal_bests":[{"workout_id":616209211,"sport":0,"date":"2015-10-08T15:33:22.000Z","type":0,"value":3.7456583568978474},{"workout_id":519068849,"sport":0,"date":"2015-05-09T13:42:37.000Z","type":1,"value":13.304793556531271},{"workout_id":112691340,"sport":0,"date":"2012-11-30T06:58:34.000Z","type":6,"value":128.46153846153845},{"workout_id":616209211,"sport":0,"date":"2015-10-08T15:33:22.000Z","type":9,"value":576.6676493658944},{"workout_id":616209211,"sport":0,"date":"2015-10-08T15:33:22.000Z","type":10,"value":961.1127489431573},{"workout_id":519068849,"sport":0,"date":"2015-05-09T13:42:37.000Z","type":11,"value":2617.283824524703},{"workout_id":519068849,"sport":0,"date":"2015-05-09T13:42:37.000Z","type":14,"value":5838.099188565341},{"workout_id":816295348,"sport":0,"date":"2016-10-02T08:00:17.000Z","type":17,"value":13561.315906179458},{"workout_id":112691340,"sport":0,"date":"2012-11-30T06:58:34.000Z","type":20,"value":206.73880615384616},{"workout_id":616209211,"sport":0,"date":"2015-10-08T15:33:22.000Z","type":22,"value":928.056621501106},{"workout_id":309007563,"sport":2,"date":"2014-02-01T10:40:40.000Z","type":1,"value":11.119920268654823},{"workout_id":217614146,"sport":9,"date":"2013-07-17T14:30:00.000Z","type":1,"value":1.5},{"workout_id":217614146,"sport":9,"date":"2013-07-17T14:30:00.000Z","type":6,"value":2400.0},{"workout_id":218062152,"sport":9,"date":"2013-07-18T14:00:00.000Z","type":10,"value":16500.0},{"workout_id":217614146,"sport":9,"date":"2013-07-17T14:30:00.000Z","type":20,"value":3862.4256},{"workout_id":210309093,"sport":18,"date":"2013-07-01T06:20:00.000Z","type":0,"value":1.481142857142857},{"workout_id":210761861,"sport":18,"date":"2013-07-02T07:05:00.000Z","type":1,"value":2.816},{"workout_id":270824758,"sport":18,"date":"2013-11-20T07:25:00.000Z","type":6,"value":483.05657211809233},{"workout_id":210309093,"sport":18,"date":"2013-07-01T06:20:00.000Z","type":9,"value":1458.3333333333333},{"workout_id":266883103,"sport":18,"date":"2013-11-08T12:00:00.000Z","type":10,"value":11250.0},{"workout_id":207470716,"sport":18,"date":"2013-06-24T13:52:46.000Z","type":11,"value":24029.234737747203},{"workout_id":210309093,"sport":18,"date":"2013-07-01T06:20:00.000Z","type":20,"value":782.32},{"workout_id":266883103,"sport":18,"date":"2013-11-08T12:00:00.000Z","type":22,"value":10863.072},{"workout_id":205407373,"sport":19,"date":"2013-06-19T07:30:00.000Z","type":1,"value":6.49}],"name":"Britt Drud Sørensen","viewer_friendship":1}
```